<a href="https://colab.research.google.com/github/Jason-Farnsworth/Computer-Graphics-Final/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np

# v, a, and b are 3-tuples; s is scalar

#return a dot b
def dot(a,b):
  return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

#return a cross b
def cross(a,b):
  return (a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0])

#return magnitude of v
def mag(v):
  return math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

#return normalized v
def normalize(v):
  m = mag(v)
  return (v[0]/m, v[1]/m, v[2]/m)

#return a - b
def sub(a,b):
  return (a[0]-b[0],a[1]-b[1],a[2]-b[2])

#return a + b
def add(a,b):
  return (a[0]+b[0],a[1]+b[1],a[2]+b[2])

#return a / s
def div(a,s):
  return (a[0]/s, a[1]/s, a[2]/s)

#return a * s
def mult(a,s):
  return (a[0]*s, a[1]*s, a[2]*s)

In [2]:
def sphere_intersections(origin, direction, spheres):
      camera = origin
      d = direction
      closest_sphere = None #closest sphere is none as of now, but will be assigned if a sphere is found (if multiple are found, this is the closest one)
      min_intersect = math.inf #Infinity
      sky_color = (146,176,202) #sky color
      pixel_color = sky_color
      #break
      for s in spheres:
        C = s.center
        r = s.radius
        color = s.color
        C = np.array(C)
        #pixel_color = (255, 255, 255)
        # solve for t by
        b = 2 * dot(d, camera - C)
        c = (mag(camera - C))** 2 - r**2 #used np.linalg.norm to get the magnitude of the vector camera -C
        delta = b**2 - 4*c
        #print(delta)
        if delta > 0:
          t1 = (-b + math.sqrt(delta)) / 2
          t2 = (-b - math.sqrt(delta)) / 2
          intersect = min(t1, t2)
          if intersect > 0 and intersect < min_intersect:
            min_intersect = intersect
            closest_sphere = s
            pixel_color = closest_sphere.color
      return closest_sphere, min_intersect, pixel_color

def triangle_intersection(origin, direction, tris):
  camera = origin
  d = direction
  sky_color = (146,176,202) #sky color
  pixel_color = sky_color
  closest_tri = None
  min_t = math.inf
  for render in tris:
      a = render.v0
      #magnitude = np.linalg.norm(np.cross((tris[1] - tris[0]), (tris[2] - tris[0]))) #is the magnitude "np.cross((tris[1] - tris[0]), (tris[2] - tris[0])" or "(tris[1] - tris[0]) - (tris[2] - tris[0])"?
      n = normalize(cross((sub(render.v1, render.v0)), (sub(render.v2, render.v0))))
      if dot(d, n) == 0: #Either we missed the triangle or are parallel to the plane, either way, we move on
        continue
      else: #Otherwise, Inside-Outside Test
        t = dot((sub(a, camera)), n) / dot(d, n)
        p = add(camera, mult(d, t))
        c0 = normalize(cross((sub(render.v1, render.v0)), (sub(p, render.v0))))
        dummy = dot(c0, n)
        if t < min_t and t > 0: #If the current is less than minimum t, then we check via the inside outside test.
        #If it's not less than min_t, then we don't care if the current t passes the inside outside test because it will never been shown
          if (dot(c0, n)) <= 0:
            continue
          c1 = normalize(cross((sub(render.v2, render.v1)), (sub(p, render.v1))))
          if (dot(c1, n)) <= 0:
            continue
          c2 = normalize(cross((sub(render.v0, render.v2)), (sub(p, render.v2))))
          if (dot(c2, n)) <= 0:
            continue
          min_t = t #if it passes all tests, then we overwrite min_t and closest_tri
          closest_tri = render
          pixel_color = closest_tri.color
  return closest_tri, min_t, pixel_color

def ray(e, d, spheres, tris):
  camera = e
  sky_color = (146,176,202) #sky color
  pixel_color = sky_color
  closest_sphere, min_intersect_sphere, pixel_color_sphere = sphere_intersections(camera, d, spheres)
  closest_tri, min_intersect_tri, pixel_color_tri = triangle_intersection(camera, d, tris)
  if min_intersect_tri < min_intersect_sphere and closest_tri is not None:
    closest_obj = closest_tri
    min_intersect = min_intersect_tri
    n = closest_tri.normal
  elif closest_sphere is not None:
    closest_obj = closest_sphere
    min_intersect = min_intersect_sphere
    p = add(camera, mult(d, min_intersect)) #3D Coordinate
    n = normalize(sub(p, closest_obj.center)) #surface normal
  else:
    closest_obj = None
    return pixel_color

  if closest_obj is not None:
    pixel_color = np.array((closest_obj.color))
    p = add(camera, mult(d, min_intersect)) #3D Coordinate
    L = normalize(light - p)
    if closest_obj.reflection > 0:
      r = sub(d, mult(mult(n, (dot(d, n))), 2))
      offset = add(p, (mult(r, 0.0001)))
      reflected_color = ray(offset, r, spheres, tris)
      temp1 = mult(closest_obj.color,(1 - closest_obj.reflection))
      temp2 = mult(reflected_color, closest_obj.reflection)
      pixel_color = np.array(add(temp1, temp2))
    blocked_obj, blocked_intersect, pixel_color_dummy = sphere_intersections(add(p, (mult(L, 0.0001))), L, spheres)
    if blocked_obj is None:
      if closest_obj.DisableShading == True:
       return pixel_color
      h = max(0, np.dot(n, L))
      #pixel_color = np.array((closest_obj.color))
      black = np.array((0,0,0))
      return_color = (pixel_color * h) + (black * (1-h))
      return return_color
    else:
      pixel_color = shadow
      return_color = pixel_color
      return return_color
  else:
    return_color = pixel_color
    return return_color
  return sky_color

In [ ]:
camera = np.array((0,0,0)) # defining the camera's coordinates
class Sphere:
  def __init__(self, center, radius, color, reflection, DisableShading = False):
    self.center = center
    self.radius = radius
    self.color = color
    self.reflection = reflection
    self.DisableShading = DisableShading

class Tri:
  def __init__(self, v0, v1, v2, color, reflection, DisableShading = False):
    self.v0 = v0
    self.v1 = v1
    self.v2 = v2
    self.color = color
    self.reflection = reflection
    self.normal = normalize(cross(sub(v1, v0), sub(v2, v0)))
    self.DisableShading = DisableShading

spheres = [
    #center coordinates, radius value, color value, reflection value
    Sphere((-5.7, 12.6, 26), .4, (0,0,0), 0, False), #left eye of 1st snowman
    Sphere((-3.6, 12.6, 26), .4, (0,0,0), 0, False), #right eye of 1st snowman
    Sphere((-5, 12.5, 29), 3, (255,255,255), 0, False), #head of 1st snowman
    Sphere((-5, 6, 30), 4, (255,255,255), 0, False), #torso of 1st snowman
    Sphere((-5, -2, 30), 5, (255,255,255), 0, False), #base of 1st snowman
    Sphere((8.9, 4.8, 35.3), .2, (0,0,0), 0, False), #left eye of 2nd snowman
    Sphere((10, 4.8, 34.84), .2, (0,0,0), 0, False), #right eye of 2nd snowman
    Sphere((10, 4.4, 37), 2, (255,255,255), 0, False), #head of 2nd snowman
    Sphere((10, .5, 37), 2.5, (255,255,255), 0, False), #torso of 2nd snowman
    Sphere((10, -3.5, 37), 3, (255,255,255), 0, False), #base of 2nd snowman
    Sphere((12, 13.5, 30), 2, (254,220,5), 0, True) #Sun
]
tris = [
    Tri((-5, 11 ,26), (-4, 11.2, 26), (-5, 11.4, 26), (237, 145, 33), 0, True), #carrot nose for 1st snowman
    Tri((9.2, 3.8, 35), (10, 4.2, 35), (9.2, 4.4, 35), (237, 145, 33), 0, True), #carrot nose for 2nd snowman
    Tri((0, -6, -5), (-10000, -6, 50), (10000, -6, 50), (148,242,244), .8, True), #floor of scene
    Tri((0, -7, -5), (-10000, -7, 10000), (10000, -7, 10000), (236,255,253), 0, True) #snowy backdrop
]


height = 2000
width = 2000
light = np.array((0, 0, 0))
screen = np.zeros((height, width, 3))
focal = math.sqrt(3)
h = 0
shadow = np.array((0,0,100))

for x in range(height):
  for y in range(width):
    sx = y * (2/width) - 1
    sy = -1 * (x * (2/height) - 1)
    s = np.array([sx, sy, focal])
    v = s - camera
    d = normalize(v)

    screen[x, y] = ray(camera, d, spheres, tris)

In [ ]:
plt.imshow((screen).astype(np.uint8)) #https://stackoverflow.com/questions/49643907/clipping-input-data-to-the-valid-range-for-imshow-with-rgb-data-0-1-for-floa
#Why it will show an error if we don't have ".astype(np.uint8)"
plt.axis('off')
plt.show()